### Engineer growing season climate data for a given crop x treatment 

In [ ]:
import pandas as pd 
import os
from IIASA_22_fxns import split_data, get_season_info, get_gs_climate

In [ ]:
#### DATA SUBSETS TO RUN REGRESSION ON

# identify data subsets to start with 
C = ["WRYE"] 
N = ["BAU", "N01", "N50", "N100", "N250"]
R = ["R00", "R30", "R60", "R90"]

# pull arguments to feed into function 
season_info_args = []
for c in C:
    for n in N: 
        if n != "N01":
            for r in R: 
                season_info_args.append((c,n,r))
        else:
            # N01 only has R00 scenario for all crops
            season_info_args.append((c,n,"R00"))

In [ ]:
#### GET SEASON INFO FOR EACH TREATMENT

season_info_dfs = []
for arg in season_info_args: 
    season_info_dfs.append(get_season_info(arg))
    print (arg, "season info, done.")

In [ ]:
####SPLIT INTO SEASON-SPECIFIC DATAFRAMES

#### Split seasonal-info dataframes into smaller dataframes where year, season length, and season start are unique
#### output is a list of lists, first dimension is treatment season info df, second dimension is season start/len split
split_dfs = []
for df in season_info_dfs:
    # add lists to split_df list that come out of split_data
    split_dfs.append(split_data(df))

In [ ]:
#### IMPORT CLIMATE FILES 
path = "//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//_Weather"
# Import all weather data files
pet_df = pd.read_csv(os.path.join(path, "CORN_dyn_rf_BAU_R00_PET.txt"), sep = ",")
prcp_df = pd.read_csv(os.path.join(path, "CORN_dyn_rf_BAU_R00_PRCP.txt"), sep = ",")
rad_df = pd.read_csv(os.path.join(path, "CORN_dyn_rf_BAU_R00_RAD.txt"), sep = ",")
tmean_df = pd.read_csv(os.path.join(path, "CORN_dyn_rf_BAU_R00_TMEAN.txt"), sep = ",")
vpd_df = pd.read_csv(os.path.join(path, "CORN_dyn_rf_BAU_R00_VPD.txt"), sep = ",")
cmd_df = pd.read_csv(os.path.join(path, "CORN_dyn_rf_BAU_R00_CMD.txt"), sep = ",")

In [ ]:
#### GROWING SEASON VARIABLE CALCULATION
# For the split up seasonal info dataframes, calculate growing season variables for a given climate variable 
climate_dfs = [pet_df, prcp_df, rad_df, tmean_df, vpd_df, cmd_df]

# save growing season climate dataframes for each treatment 
GS_dfs = []
# loop over the split dataframes of all treatments
for i in range(len(split_dfs)):
    # save growing season climate for each climate variable
    GSclim_dfs = []
    # loop over all climate variable
    for clim_df in climate_dfs: 
        clim_out_list = []
        # go over all the season len splits in a given treatment and get the climate cols for the seasonal split 
        for df in split_dfs[i]:
            # get season x treatment specific engineered variables
            clim_out_list.append(get_gs_climate((df, clim_df)))
        # for all the splits, concatenate them vertically then add to the list
        GSclim_dfs.append(pd.concat(clim_out_list))
        print ('clim done')
    # drop duplicate variables from new dataframes 
    concat_clim = pd.concat(GSclim_dfs, axis = 1)
    ready = concat_clim.loc[:,~concat_clim.columns.duplicated()]
    
    GS_dfs.append(ready)
    print ("treatment done.")

In [ ]:
# list of growing season climate file names
clim_df_names = [arg[0] + "_" + arg[1] + "_" + arg[2] + "_GSclim.csv" for arg in season_info_args]

In [ ]:
# save growing season climate dataframes
path = "//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//WRYE"
for i in range(len(GS_dfs)): 
    out_file =  os.path.join(path, clim_df_names[i])
    GS_dfs[i].to_csv(out_file)